In [11]:
import numpy as np 
import pandas as pd 
import requests
import xlsxwriter
import math

In [2]:
# Importing our list of stocks 
stocks = pd.read_csv('sp_500_stocks.csv')

In [3]:
# Acquiring API token 
from secrets import IEX_CLOUD_API_TOKEN

In [4]:
# Using a Batch API call to create the dataframe 
def chunks(lst, n): 
    for i in range(0, len(lst), n): 
        yield lst[i:i + n] 

In [5]:
columns = ['Ticker', 'Stock Price', 'Market Cap', 'Number of Shares to Buy']

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
final_dataframe = pd.DataFrame(columns = columns)
for i in range(0, len(symbol_groups)): 
    symbol_strings.append(','.join(symbol_groups[i]))

In [6]:
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}' 
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
            [
                symbol, 
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = columns
        ), 
        ignore_index=True
    )


In [7]:
final_dataframe.tail()

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
500,YUM,106.911,31388329489,N/A
501,ZBH,160.860,34187250355,N/A
502,ZBRA,415.580,22799923710,N/A
503,ZION,50.540,7933713449,N/A
504,ZTS,164.250,75551103447,N/A


In [8]:
# User inputs Portfolio Value
portfolio_size = input('Enter the Value of Portfolio: ')

try: 
    val = float(portfolio_size)
except ValueError: 
    print('Please enter an integer')
    portfolio_size = input('Enter the Value of Portfolio: ')
    val = float(portfolio_size)

In [9]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)): 
    final_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Cap,Number of Shares to Buy
0,A,129.760,38476713406,152
1,AAL,17.600,10837960198,1125
2,AAP,158.830,11163871241,124
3,AAPL,142.430,2321983259205,139
4,ABBV,111.920,193717400000,176
...,...,...,...,...
500,YUM,106.911,31388329489,185
501,ZBH,160.860,34187250355,123
502,ZBRA,415.580,22799923710,47
503,ZION,50.540,7933713449,391


In [14]:
# Outputing report to Excel 
writer = pd.ExcelWriter('recommended trades.xlsx')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

In [17]:

background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [19]:
# Saving Excel Output 
writer.save()